In [1]:
import csv
import pickle
from collections import defaultdict

Получение списка игроков с числом очков больше 10000 (/ 7500 / 5000)

In [2]:
map_points = {}
with open('data/maps.csv', 'r', encoding='latin-1') as fp:
    reader = csv.reader(fp, delimiter=',')
    next(reader)
    for row in reader:
        map_points[row[0]] = int(row[2])
print(len(map_points))


2025


In [3]:
players = defaultdict(set)
with open('data/race.csv', 'r', encoding='latin-1') as fp:
    reader = csv.reader(fp, delimiter=',')
    next(reader)
    for row in reader:
        if len(row) == 5:
            players[row[1]].add(row[0])
        elif len(row) == 4:
            players[row[1].split('",')[0]].add(row[0])
        elif len(row) == 6:
            players[row[1] + row[2]].add(row[0])
        elif len(row) == 3:
            players[row[1].split('",')[0]].add(row[0])
print(len(players))


459661


In [4]:
selected_players = dict()
for player in players.keys():
    pts = 0
    for map_name in players[player]:
        pts += map_points[map_name]
    if pts >= 7500:
        selected_players[player] = pts

print(selected_players['PeX'])
print(selected_players['Cor'])
del map_points
del players

11317
27795


Получение списка сокомандников

In [5]:
def extraction(spl_line):
    if len(spl_line) != 3:
        print('wrong line!')
        return None
    else:
        mapname, date = spl_line[0][1:], spl_line[2][:-1]
        splt_l = spl_line[1].split('",')
        if len(splt_l) == 2:
            nick, sp_l = splt_l
            time, team_id = sp_l.split(',"')
        else:
            nick, time, team_id = splt_l
        if ',' in time:
            time, id_reminder = time.split(',')
            if id_reminder:
                team_id = id_reminder + team_id
        return mapname, nick, time, team_id, date


In [6]:
record_list = []

prev_line = []
with open('data/teamrace.csv', 'r', encoding='latin-1') as f:
    next(f)
    for line in f:
        splitted_line = line.split('","')
        if len(splitted_line) == 3:
            record_list.append(extraction(splitted_line))
            prev_line = splitted_line
        else:
            if len(splitted_line) == 2 and len(prev_line) == 2:
                new_line = [prev_line[0], prev_line[1] + splitted_line[0], splitted_line[1]]
                record_list.append(extraction(new_line))
                prev_line = new_line
            else:
                if len(splitted_line) == 1 and len(prev_line) == 2:
                    prev_line = [prev_line[0], prev_line[1] + splitted_line[0]]
                else:
                    prev_line = splitted_line
len(record_list)

1274201

In [7]:
teamid_dict = defaultdict(set)
for row in record_list:
    teamid_dict[row[3]].add(row[1])

print(len(teamid_dict))
del record_list

587865


In [8]:
team_mates = dict()
sp = selected_players.keys()

for tid in teamid_dict.keys():
    for player in teamid_dict[tid]:
        for mate in teamid_dict[tid]:
            if player in sp and mate in sp:
                if player not in team_mates.keys():
                    team_mates[player] = dict()
                if mate not in team_mates[player].keys():
                    team_mates[player][mate] = 1
                else:
                    team_mates[player][mate] += 1
for k in list(team_mates.keys()):
    del team_mates[k][k]
    if not team_mates[k]:
        del team_mates[k]

print(len(team_mates))
del teamid_dict

520


In [9]:
def get_edge_weight(t_c):
    w = 0
    if t_c == 1:
        w = 0.05
    if 2 <= t_c <= 3:
        w = 0.075
    if 4 <= t_c <= 5:
        w = 0.1
    if 6 <= t_c <= 10:
        w = 0.2
    if 11 <= t_c <= 20:
        w = 0.3
    if 21 <= t_c <= 50:
        w = 0.4
    if 51 <= t_c <= 100:
        w = 0.5
    if 101 <= t_c <= 200:
        w = 0.7
    if 201 <= t_c <= 300:
        w = 0.8
    if 301 <= t_c <= 500:
        w = 0.9
    if 501 <= t_c:
        w = 1
    return w

In [10]:
max_pts = max(selected_players.values())
print('max_pts:', max_pts)

nodes_list = []
id_player = dict()
for i, k in enumerate(team_mates.keys()):
    id_player[k] = i
    nodes_list.append((i, {'labels': k, 'weight': selected_players[k] / max_pts}))

print(len(nodes_list))
del selected_players

max_pts: 27795
520


In [11]:
edges = []
for k in team_mates.keys():
    for m in team_mates[k].keys():
        edges.append((id_player[k], id_player[m], get_edge_weight(team_mates[k][m])))

print(len(edges))
del team_mates

21574


In [12]:
print(len(edges))
for i, e in enumerate(edges):
    if i == 10:
        break
    print(e)

21574
(0, 6, 0.3)
(0, 14, 0.4)
(0, 13, 0.05)
(0, 28, 0.075)
(0, 26, 0.05)
(0, 7, 0.1)
(0, 21, 0.05)
(0, 70, 0.05)
(0, 47, 0.05)
(0, 8, 0.075)


In [13]:
# with open('output/edges_short.csv', 'w', encoding='latin-1') as f:
#     f.write("v1,v2,wt\n")
#     for v1, v2, wt in edges:
#         f.write('%d,%d,%f\n' % (v1, v2, wt))

In [14]:
with open('output/nodes.pickle', 'wb') as f:
    pickle.dump(nodes_list, f)

with open('output/edges.pickle', 'wb') as f:
    pickle.dump(edges, f)
